In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [4]:
# 删除空数据
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [5]:
## 创建dataset
from torch.utils.data import Dataset

class MyDataSet(Dataset):
  def __init__(self) -> None:
    super().__init__()
    self.data = pd.read_csv("/content/drive/MyDrive/ChnSentiCorp_htl_all.csv")
    self.data = self.data.dropna()

  def __getitem__(self, index):
    return self.data.iloc[index]["review"], self.data.iloc[index]["label"] # data.iloc[index]取DataFrame中第index行的数据(从0开始)

  def __len__(self):
    return len(self.data)

In [6]:
dataset = MyDataSet()
for i in range(5):
  print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', np.int64(1))
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', np.int64(1))
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', np.int64(1))
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', np.int64(1))
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', np.int64(1))


In [7]:
# 划分数据集
from torch.utils.data import random_split
trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [8]:
for i in range(10):
    print(trainset[i])

('中庸的酒店,本来冲着环保房去的,可是,哎,有些失望啊', np.int64(0))
('我非常不满意新月阁客栈，如果有零分可以选，我宁可一分也不肯给.他们的服务太差了.明明含早餐的,居然早上服务员全然不知道,还是我本人大清早冒雨出去买的。更让人心寒和气愤的是:在中秋节的早晨,服务员在我们毫不知情的情况下,把房间转卖给别人,把我和一对美国夫妇赶走了。我和周围的几家客栈谈起此事,大家都非常震惊,居然有这种人!我强烈建议大家从此以后不要在这里住了。而且价格是附近的客栈的两倍.', np.int64(0))
('不知怎么回事，一直想给泰安华侨大厦点评，却一直未成功。因为酒店的服务很好。前一段时间和同事一起去泰安爬泰山，晚上12点出发，前台的服务员告诉我们山上气温低，让我们多加件衣服。挺细心的。因为凌晨退房，走的急，钱包落在床下了。快到山脚时，酒店的服务人员给我们打电话，说捡到我们的钱包了。我们这才发现钱包丢了。当即回到酒店，记得那位值班人员叫苏秀，我们在此表示感谢。谢谢！宾馆反馈2008年6月1日：我们酒店是老牌四星级酒店，员工素质高，个个都有十几年的服务经验，对客服务在泰城数首屈一值，宾客选择我们酒店会住得温馨、安心、放心的。', np.int64(1))
('和父母一起去的，所以住在家庭房里，没有看到无敌江景。总得来说，酒店弄得比较干净。韩国人特别多。但是房间里老是有一股去不了的怪味道，马桶偶尔漏漏水，莲蓬头设计得非常容易往外溅水。早餐也不咋地。虽然缺点多多，但是已经算是丹东最顶尖的酒店了。', np.int64(1))
('这个价位住到这样的房间对我来说是惊喜，订的商务大床房，很夸敞舒适，房间最里面顺着靠窗的地方走进去居然是个小小的书房，很不错！', np.int64(1))
('订的是山景房,房间还不错,挺干净.前台服务员有点小问题：一是人数偏少，只有两个人，另外最好多点笑容；二是前台服务员对本地的景点不熟悉是最大的遗憾了。', np.int64(1))
('酒店除了所处位置非常有优势外，其他的应该没什么了。设施简单、陈旧，但对于住一晚的我来说，也能将就。', np.int64(1))
('不错的酒店,服务还可以,下次还会入住的~', np.int64(1))
('早餐不好,服务不到位,晚餐无西餐,早餐晚餐相同,房间条件不好,餐厅不分吸烟区.房间不分有无烟房.'

In [9]:
# 创建 Dataloader
import torch
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
  texts, labels = [], []
  # batch 是一个列表，其中每个元素是你 Dataset 的 __getitem__ 返回的内容
  # batch = [ ("酒店很好", 1), ("服务很差", 0), ("一般般", 1) ]
  for item in batch:
    texts.append(item[0])
    labels.append(item[1])
  inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt") # truncation=True,超过 128 的截断
  inputs["labels"] = torch.tensor(labels)
  return inputs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hugging Face 的 Trainer 或很多自定义模型都期望输入是一个包含 labels 键的字典

collate_func()返回的是：

{

  'input_ids': ...,

  'attention_mask': ...,

  'labels': tensor([1, 0, 1])
  
}

In [10]:
from torch.utils.data import DataLoader
# shuffle在每个 epoch 开始时，打乱 trainset 中样本的读取顺序，让模型在每个 epoch 看到不同的 batch 顺序，增强泛化能力，避免优化陷入局部模式
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [11]:
next(enumerate(validloader))[1] # 作用是从 validloader（一个 PyTorch 的 DataLoader 对象）中取出第一个 batch 的数据。

{'input_ids': tensor([[ 101, 1282, 3613,  ..., 1565, 4638,  102],
        [ 101, 1963, 3362,  ..., 2600,  722,  102],
        [ 101, 2769, 1372,  ..., 2552,  119,  102],
        ...,
        [ 101, 2697, 6230,  ..., 8024, 4294,  102],
        [ 101, 1765, 4415,  ...,    0,    0,    0],
        [ 101, 2145, 2791,  ..., 2157, 1724,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1

In [12]:
# 创建模型及优化器
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

pytorch_model.bin:   0%|          | 0.00/156M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
optimizer = Adam(model.parameters(), lr=2e-5)

In [14]:
# 训练和验证
def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

In [15]:
!nvidia-smi

Thu Nov 27 02:16:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             25W /   70W |     266MiB /  15360MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
train()

ep: 0, global_step: 0, loss: 0.6702806353569031
ep: 0, global_step: 100, loss: 0.39883601665496826
ep: 0, global_step: 200, loss: 0.3736916780471802
ep: 0, acc: 0.8827319145202637
ep: 1, global_step: 300, loss: 0.2907899022102356
ep: 1, global_step: 400, loss: 0.2423524558544159
ep: 1, acc: 0.8904638886451721
ep: 2, global_step: 500, loss: 0.0745791643857956
ep: 2, global_step: 600, loss: 0.055822696536779404
ep: 2, acc: 0.8917525410652161


In [17]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [18]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)

Device set to use cuda:0


[{'label': '好评！', 'score': 0.9905791878700256}]